In [ ]:
from legion.metrics import send_metric, Metric
from legion.model.model_id import init

In [ ]:
init('test summation')

In [ ]:
import numpy as np
import pandas as pd
input_data = {'a': [10, 20], 'b': [20, 30]}
df = pd.DataFrame(input_data)

In [ ]:
def calculate(x):
    return x['a'] + x['b']

In [ ]:
send_metric(Metric.TEST_ACCURACY, 100.0)

In [ ]:
df.dtypes

In [ ]:
import legion.io

legion.io.export(apply_func=lambda x : {'result': int(calculate(x)) },
                 input_data_frame=df, 
                 use_df=False,
                 version='1.0')